## Imports

In [1]:
import pyxdf 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from lmfit.models import Model
from os import listdir, getcwd
from os.path import isfile, join

## Function Definitions


Predicted pupil dilation, $d(Y)$, caused by luminance $Y$, is computed with the following equation: $𝑑(𝑌) = 𝑎 · 𝑒^{−𝑏·𝑌} + c$

In [2]:
def pupil_func(x, a, b, c):
    return a * np.exp(-b * x) + c

In [3]:
def import_data(file):
    streams, header = pyxdf.load_xdf(file)
    dfs = {}
    for stream in streams:
        stream_name = stream['info']['name'][0]
        stream_channels = {channel['label'][0]: i for i, channel in enumerate(stream['info']['desc'][0]['channels'][0]['channel'])}
        stream_data = stream['time_series']
        data_dict = {key: np.array(stream_data)[:, index] for key, index in stream_channels.items()}
        data_dict['time'] = np.around(np.array(stream['time_stamps']), decimals=4)
        dfs[stream_name] = pd.DataFrame(data_dict).drop_duplicates(subset=['time']).reset_index(drop=True)

## Import Data

In [4]:
data_dir = join(getcwd(),'Path_Data')
data_files = [join(data_dir, f) for f in listdir(data_dir) if isfile(join(data_dir, f))]

In [5]:
dfs = {}
for i, file in enumerate(data_files):
    dfs[i] = import_data(file)

In [ ]:
accom_time = pd.to_timedelta(0.5, unit='s')

pupil = dfs['GazeStream'].loc[(dfs['GazeStream']['LeftEyeIsBlinking'] == 0) & (dfs['GazeStream']['RightEyeIsBlinking'] == 0) & (dfs['GazeStream']['LeftPupilDiameter'] > 0) & (dfs['GazeStream']['RightPupilDiameter'] > 0), ['time', 'MethodID', 'ModelID', 'LeftPupilDiameter', 'RightPupilDiameter']]
pupil['time'] = pd.to_timedelta(pupil['time'], unit='s')

lum = dfs['LuminanceStream'].loc[:, ['time', 'MethodID', 'ModelID', 'Luminance']]
lum['time'] = pd.to_timedelta(lum['time'], unit='s')

# Intersection of time stamps
pupil_lum_time_intersection = np.intersect1d(pupil['time'], lum['time'])

# Filter pupil and luminance data by intersection
pupil = pupil[pupil['time'].isin(pupil_lum_time_intersection)].reset_index(drop=True)
lum = lum[lum['time'].isin(pupil_lum_time_intersection)].reset_index(drop=True)

# Combined DataFrame for pupil and luminance
pup_lum_df = pd.DataFrame({
    'time': pd.to_timedelta(lum['time'], unit='s'),
    'luminance': lum['Luminance'],
    'pupilDiameter': 0.5 * (pupil['LeftPupilDiameter'] + pupil['RightPupilDiameter']),
    'methodID': pupil['MethodID'],
    'modelID': pupil['ModelID']
}).resample('0.1s', on='time').mean()

pup_lum_df['time'] = pup_lum_df.index

# Calibration data 
calibration_events = dfs['ExperimentStream'].loc[(dfs['ExperimentStream']['EventType'] == 'CalibrationColorChange') | (dfs['ExperimentStream']['SceneEvent'] == 'Calibration') | (dfs['ExperimentStream']['SceneEvent'] == 'CalibrationComplete'), ['time','SceneEvent', 'EventType']]
calibration_events['time'] = pd.to_timedelta(calibration_events['time'], unit='s')

start_times = calibration_events[:8]['time']
end_times = calibration_events[1:]['time']
start_times = start_times.reset_index(drop=True)
end_times = end_times.reset_index(drop=True)

pupil_calib = pup_lum_df.loc[(pup_lum_df['methodID'] > 98.0), ['pupilDiameter']]
lum_calib = pup_lum_df.loc[(pup_lum_df['methodID'] > 98.0), ['luminance']]

calib_data = {}
for i in range(8):
    calib_data[i] = pup_lum_df.loc[(pup_lum_df['time'] >= start_times[i]) & (pup_lum_df['time'] <= end_times[i]), ['time','luminance', 'pupilDiameter']]
    calib_data[i]['time'] -= calib_data[i]['time'].iloc[0]
    calib_data[i] = calib_data[i].loc[(calib_data[i]['time'] >= accom_time), ['luminance', 'pupilDiameter']]

calibration_data = pd.concat(calib_data).groupby(level=0).mean().sort_values(by=['luminance']).reset_index(drop=True)